In [1]:
import pandas as pd
import sqlite3

# Load the data
file_path = 'Pokemon.csv'  # Update to your actual file path
pokemon = pd.read_csv(file_path)
pokemon.head()

,number,name,type1,type2,total,hp,attack,defense,sp_attack,sp_defense,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,3,Gigantamax Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False


In [2]:
pokemon['type1'].unique()

array(['Grass', 'Fire', 'Water', 'Blastoise', 'Bug', 'Normal', 'Dark',
       'Poison', 'Electric', 'Ground', 'Ice', 'Fairy', 'Steel',
       'Fighting', 'Psychic', 'Rock', 'Ghost', 'Dragon', 'Flying',
       'Graass'], dtype=object)

In [3]:
#fixing typos in type1 column
pokemon.loc[pokemon["type1"] == "Graass", "type1"] = "Grass"
pokemon.loc[pokemon['type1'] == 'Blastoise', 'type1'] = 'Water'

In [4]:
# Fill missing values in Type 2 with Type 1
pokemon['type2'] = pokemon['type2'].fillna(pokemon['type1'])

In [5]:
pokemon['generation'].unique()

array([1, 7, 8, 2, 3, 4, 5, 6, 0], dtype=int64)

In [6]:
#there is no generation zero in pokemon !?
pokemon[pokemon['generation'] == 0]

,number,name,type1,type2,total,hp,attack,defense,sp_attack,sp_defense,speed,generation,legendary
950,808,Meltan,Steel,Steel,300,46,65,65,55,35,34,0,True
951,809,Melmetal,Steel,Steel,600,135,143,143,80,65,34,0,True
952,809,Gigantamax Melmetal,Steel,Steel,600,135,143,143,80,65,34,0,True


In [7]:
#meltan pokemon started from gen VII
pokemon.loc[pokemon['generation'] == 0, 'generation'] = 7
pokemon['generation'].unique()

array([1, 7, 8, 2, 3, 4, 5, 6], dtype=int64)

In [8]:
# Split into original forms and other forms
original_forms = pokemon.drop_duplicates(subset="number", keep="first")
other_forms = pokemon[pokemon.duplicated(subset="number", keep="first")]

In [9]:
# Create SQLite database and save tables
db_name = 'pokemon.db'
connection = sqlite3.connect(db_name)

original_forms.to_sql('original_forms', connection, if_exists='replace', index=False)
other_forms.to_sql('other_forms', connection, if_exists='replace', index=False)

174

In [10]:
# Filtering Function
def filter_legendary(table_name, is_legendary=True):
    """Filter Pokémon based on their legendary status from a specific table."""
    query = f"""
    SELECT * 
    FROM {table_name}
    WHERE legendary = {1 if is_legendary else 0};
    """
    return pd.read_sql_query(query, connection)

# Example Usage: Filter Legendary Pokémon
legendary_original = filter_legendary('original_forms', is_legendary=True)
legendary_other = filter_legendary('other_forms', is_legendary=True)

# Close the database connection
connection.close()

In [11]:
# Display results for verification
print("Legendary Pokémon (Original Forms):")
print(legendary_original.head())

Legendary Pokémon (Original Forms):
   number      name     type1    type2  total   hp  attack  defense  \
0     144  Articuno       Ice   Flying    580   90      85      100   
1     145    Zapdos  Electric   Flying    580   90      90       85   
2     146   Moltres      Fire   Flying    580   90     100       90   
3     150    Mewtwo   Psychic  Psychic    680  106     110       90   
4     151       Mew   Psychic  Psychic    600  100     100      100   

   sp_attack  sp_defense  speed  generation  legendary  
0         95         125     85           1          1  
1        125          90    100           1          1  
2        125          85     90           1          1  
3        154          90    130           1          1  
4        100         100    100           1          1  


In [12]:
print("\nLegendary Pokémon (Other Forms):")
print(legendary_other.head())


Legendary Pokémon (Other Forms):
   number               name     type1     type2  total   hp  attack  defense  \
0     144  Galarian Articuno   Psychic    Flying    580   90      85       85   
1     145    Galarian Zapdos  Fighting    Flying    580   90     125       90   
2     146   Galarian Moltres      Dark    Flying    580   90      85       90   
3     150      Mega Mewtwo X   Psychic  Fighting    780  106     190      100   
4     150      Mega Mewtwo Y   Psychic   Psychic    780  106     150       70   

   sp_attack  sp_defense  speed  generation  legendary  
0        125         100     95           8          1  
1         85          90    100           8          1  
2        100         125     90           8          1  
3        154         100    130           1          1  
4        194         120    140           1          1  
